In [0]:
from pyspark.sql import functions as F

confusion_table = "nlp_dev.gold_gold.fact_sentiment_confusion"
summary_table = "nlp_dev.gold_gold.fact_sentiment_summary"

df_conf = spark.table(confusion_table)
df_summ = spark.table(summary_table)

display(df_conf)
display(df_summ)


In [0]:
# Take the overall metrics from summary table (it already has overall_accuracy)
df_latest_summary = df_summ.limit(1)

# Add model metadata manually for now
model_name    = "fasttext_sentiment_lr"
model_version = 1
df_metrics = (
    df_latest_summary
      .withColumn("model_name",    F.lit(model_name))
      .withColumn("model_version", F.lit(model_version))
      .withColumn("metric_scope",  F.lit("test"))
      .withColumn("created_at",    F.current_timestamp())
)

display(df_metrics)


In [0]:
metrics_path = "abfss://nlp@nlplakeadls001.dfs.core.windows.net/agents/model_metrics"

(
    df_metrics.write.format("delta").mode("append").save(metrics_path)
)


In [0]:
%sql
create table if not exists nlp_dev.agents.model_metrics
using delta
LOCATION 'abfss://nlp@nlplakeadls001.dfs.core.windows.net/agents/model_metrics';

In [0]:
%sql
select * from nlp_dev.agents.model_metrics